In [14]:
import joblib
import pandas as pd
import numpy as np
from sklearn.neighbors import BallTree

# Загружаем данные о станциях
data_cleaned = pd.read_csv('/Users/mmms/Developer/team10project/team10project/data/processed/data_cleaned.csv')

# Функция для поиска 1 ближайшей станции
def find_nearest_station(user_coords, data):
    data_coords = data[['latitude', 'longitude']].values
    user_coords = np.array(user_coords).reshape(1, -1)
    tree = BallTree(data_coords, metric='euclidean')
    dist, ind = tree.query(user_coords, k=1)  # Находим 1 ближайшую точку
    nearest_station = data.iloc[ind[0][0]]
    return nearest_station

# Функция для создания набора признаков
def create_features(user_coords, data, day_of_week):
    # Шаг 1: Найти ближайшую станцию
    nearest_station = find_nearest_station(user_coords, data)
    
    # Шаг 2: Использовать ее признаки
    features = nearest_station[['temp_mean', 'wdir_mean', 'wspd_mean', 'pm10', 'log_pm25']].to_dict()
    
    # Добавляем день недели
    features['day_of_week'] = day_of_week
    
    # Преобразуем в DataFrame для подачи в модель
    feature_df = pd.DataFrame([features])
    return feature_df

# Загрузка модели
model_path = '/Users/mmms/Developer/team10project/team10project/model/new_aqi_model.pkl'
model = joblib.load(model_path)

def create_7_day_features(user_coords, data_cleaned, current_day_of_week):
    """
    Генерация признаков для 7 дней на основе ближайшей станции.
    Добавляем изменение признаков по дням.
    """
    features_list = []
    for day_offset in range(7):  # 7 дней вперед
        # Определяем день недели (цикл 0-6)
        day_of_week = (current_day_of_week + day_offset) % 7

        # Генерация базовых признаков
        features = create_features(user_coords, data_cleaned, day_of_week)

        # Добавляем небольшие изменения (симуляция сезонности или погоды)
        features['temp_mean'] += np.random.uniform(-1, 1) * day_offset
        features['wspd_mean'] += np.random.uniform(-0.5, 0.5) * day_offset
        features['log_pm25'] += np.random.uniform(-0.1, 0.1) * day_offset

        # Удаляем 'day_of_week', так как модель не использует его
        features = features[['temp_mean', 'wdir_mean', 'wspd_mean', 'pm10', 'log_pm25']]
        
        features_list.append(features)

    # Объединяем все признаки для 7 дней
    return pd.concat(features_list, axis=0, ignore_index=True)

# Координаты пользователя и текущий день недели
current_day_of_week = 2  # Например, сегодня среда
user_coords = (55.644227769578535, 37.438901726516136)  # Пример: Москва

# Генерация признаков для 7 дней
input_features_7_days = create_7_day_features(user_coords, data_cleaned, current_day_of_week)

# Делаем предсказание на 7 дней
predicted_aqi_7_days = model.predict(input_features_7_days)

# Выводим результат
for i, aqi in enumerate(predicted_aqi_7_days):
    print(f"Прогноз качества воздуха (AQI) на день {i+1}: {aqi:.2f}")

Прогноз качества воздуха (AQI) на день 1: 35.62
Прогноз качества воздуха (AQI) на день 2: 36.87
Прогноз качества воздуха (AQI) на день 3: 39.17
Прогноз качества воздуха (AQI) на день 4: 37.91
Прогноз качества воздуха (AQI) на день 5: 33.60
Прогноз качества воздуха (AQI) на день 6: 34.12
Прогноз качества воздуха (AQI) на день 7: 33.92
